In [1]:
import os
%pwd

'/home/petar/Documents/python_projects/petar-milivojevic-mlhU-machine-learning-new-mP1l/research'

In [2]:
os.chdir('../')
%pwd

'/home/petar/Documents/python_projects/petar-milivojevic-mlhU-machine-learning-new-mP1l'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    dataset_folder: Path
    dataset_xlsx: Path
    STATUS_FILE: Path
    all_schema: list

In [12]:
from aiusage.constant import *
from aiusage.utils.main_utils import create_directories, read_yaml

class ConfigurationMananger:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            dataset_folder=config.dataset_folder,
            dataset_xlsx=config.dataset_xlsx,
            STATUS_FILE=config.STATUS_FILE,
            all_schema=schema
        )
        
        return data_validation_config

In [16]:
import pandas as pd

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
    
    def validate_dataset(self)-> bool:
        try:
            validation_status = None

            data_df = pd.read_excel(self.config.dataset_xlsx)
            all_cols = list(data_df.columns)

            for col in self.config.all_schema:
                if col not in all_cols:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                
                    return validation_status
                
                elif validation_status == None:
                    validation_status = True
            
            for name in data_df[data_df.columns[0]].unique():
                if name not in os.listdir(self.config.dataset_folder):
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                    
                    return validation_status
            
            with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                    
            return validation_status
        
        except Exception as e:
            raise e

In [15]:
try:
    config = ConfigurationMananger()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_dataset()

except Exception as e:
    raise e

[2025-01-07 20:17:00,551: INFO: main_utils: created directory at: artifacts]
[2025-01-07 20:17:00,552: INFO: main_utils: created directory at: artifacts/data_validation]
